# deltatau-audit — Quickstart

**Find and fix timing failures in RL agents.**

This notebook demonstrates the full workflow:
1. Install `deltatau-audit`
2. Run the CartPole Before/After demo (bundled checkpoints, no training needed)
3. See the Before/After comparison table

**Total runtime: ~2 minutes on CPU. No GPU needed.**

---
GitHub: [maruyamakoju/deltatau-audit](https://github.com/maruyamakoju/deltatau-audit)  
PyPI: `pip install deltatau-audit`

## Step 1: Install

In [ ]:
!pip install -q "deltatau-audit[demo]"

## What is a timing failure?

RL agents are trained at a fixed simulation speed (e.g. 50 Hz). In deployment, the actual step rate varies:
- **Speed changes**: 30 Hz → 50 Hz → 100 Hz
- **Observation delay**: sensor lag, network latency
- **Mid-episode spikes**: system load bursts

Standard PPO agents are completely unaware of this. The result:

| Scenario | Standard PPO (HalfCheetah) |
|----------|:--:|
| Observation delay (1 step) | **-96% return** |
| 5x speed (unseen) | **-109% (agent goes negative)** |
| Speed jitter | **-75%** |

`deltatau-audit` measures these failures and fixes them automatically.

## Step 2: Run the CartPole Demo

This audits two bundled checkpoints:
- **Before**: Standard GRU agent (speed-naive training)
- **After**: Speed-randomized GRU agent (trained with variable timing)

Each agent is tested under **5 conditions**: nominal speed, speed jitter, observation delay, mid-episode spike, **observation noise** (σ=0.1), and 5× stress speed.

Use `--workers auto` to parallelize across CPU cores (2–6× speedup).

In [ ]:
import subprocess
result = subprocess.run(
    ["python", "-m", "deltatau_audit", "demo", "cartpole",
     "--episodes", "20", "--out", "demo_report/",
     "--workers", "auto"],   # use all CPU cores for 2-6x speedup
    capture_output=False, text=True
)

## Step 3: View Results

In [ ]:
import json
from pathlib import Path

def load_summary(path):
    with open(path) as f:
        return json.load(f)

baseline = load_summary("demo_report/baseline/summary.json")
robust   = load_summary("demo_report/robust_wide/summary.json")

b_rob = baseline["robustness"]["per_scenario_scores"]
a_rob = robust["robustness"]["per_scenario_scores"]

print(f"{'Scenario':14s}  {'Before':>10s}  {'After':>10s}  {'Change':>10s}")
print("-" * 52)
for sc in b_rob:
    b = b_rob[sc]["return_ratio"] * 100
    a = a_rob.get(sc, {}).get("return_ratio", 0) * 100
    d = a - b
    sign = "+" if d >= 0 else ""
    print(f"{sc:14s}  {b:9.1f}%  {a:9.1f}%  {sign}{d:8.1f}pp")

print()
b_d = baseline["deployment_score"]
a_d = robust["deployment_score"]
b_r = baseline["deployment_rating"]
a_r = robust["deployment_rating"]
print(f"Deployment: {b_r} ({b_d:.2f}) → {a_r} ({a_d:.2f})")

b_s = baseline["stress_score"]
a_s = robust["stress_score"]
b_sr = baseline["stress_rating"]
a_sr = robust["stress_rating"]
print(f"Stress:     {b_sr} ({b_s:.2f}) → {a_sr} ({a_s:.2f})")

## Audit Your Own SB3 Model

```bash
pip install "deltatau-audit[sb3,mujoco]"
deltatau-audit audit-sb3 --algo ppo --model my_model.zip --env HalfCheetah-v5
```

Or via Python:

In [ ]:
# Example: audit any SB3 model
# (requires: pip install "deltatau-audit[sb3]")

# from stable_baselines3 import PPO
# from deltatau_audit.adapters.sb3 import SB3Adapter
# from deltatau_audit.auditor import run_full_audit
# from deltatau_audit.report import generate_report
# import gymnasium as gym
#
# model = PPO.load("my_model.zip")
# adapter = SB3Adapter(model)
# result = run_full_audit(
#     adapter,
#     lambda: gym.make("HalfCheetah-v5"),
#     speeds=[1, 2, 3, 5, 8],
#     n_episodes=30,
#     n_workers=4,   # parallel episode collection (2-4x speedup)
#     seed=42,       # reproducible results
# )
# generate_report(result, "my_audit/")

print("Uncomment the code above to audit your own model.")

## Find AND Fix in One Command

```bash
deltatau-audit fix-sb3 --algo ppo --model my_model.zip --env HalfCheetah-v5
```

This single command:
1. **Audits** your model (finds timing failures)
2. **Retrains** with speed randomization (the fix)
3. **Re-audits** the fixed model (verifies the fix)
4. **Generates** Before/After HTML report

Or use the Python API:

In [ ]:
# Full fix pipeline (requires stable-baselines3)

# from deltatau_audit.fixer import fix_sb3_model
# result = fix_sb3_model(
#     model_path="my_model.zip",
#     algo="ppo",
#     env_id="CartPole-v1",
#     output_dir="fix_output/",
#     timesteps=100_000,
# )
# print(f"Fixed model: {result['fixed_model_path']}")
# print(f"Before: {result['before']['summary']['deployment_rating']}")
# print(f"After:  {result['after']['summary']['deployment_rating']}")

print("Uncomment the code above to run the full fix pipeline.")

## CI/CD Integration

```bash
python -m deltatau_audit demo cartpole --ci
# exit 0 = pass, exit 1 = warn (stress), exit 2 = fail (deployment)
```

GitHub Action:
```yaml
- uses: maruyamakoju/deltatau-audit@main
  with:
    command: audit-sb3
    model: model.zip
    algo: ppo
    env: CartPole-v1
```

---

**Links:**
- [GitHub](https://github.com/maruyamakoju/deltatau-audit)
- [PyPI](https://pypi.org/project/deltatau-audit/)
- [Interactive HalfCheetah report](https://maruyamakoju.github.io/deltatau-audit/sample/halfcheetah/)